# ChatGPT를 활용한 브런치 크롤링 짜보기

### TRY 1
- 전체질문 (Basic question)
1. URL을 정의해줘야함  
2. 제목, 내용, 작성날짜 
3. 엑셀로 다운로드 되어야함
>* Q. Can you make a crawler with python selenium? url is the down below. Also, I wanna change the keyword as well. Also wanna save to excel file. keyword="강아지" , url="https://brunch.co.kr/keyword/"+f"{keyword}" and I want to crawl the titles, contents, and date infomation. the html class name is "cover_title" / "wrap_item item_type_text" / "f_l date" for each. Also I want to save to excel file. 

### TRY 2
4. URL 가지고 올때 미세조정 (scroll date기준으로 추가)
>* Q.are there any solutions that can try to stop scrolling until a specific date(ex. 2022.01.01)

- 브런치의 경우 시간이 정확히 찍히지 X (ex.1시간전, May 01 2023)
- 따라서 len으로 구분하여 date parsing (이건 본인 아이디어)
- 던진 쿼리:
>* Q. I want to subtract the current time from the Mar 27 2023 form
>* Q. I want to change string type "1 hour ago" to time
- 그러나.. scrolling 중 date를 비교할때 스크롤을 내린 만큼의 date 정보를 가져오게 되어 시간이 엄청 오래걸리게 됨
- 던진 쿼리: 
>* Q. When comparing dates during scrolling, date information as much as scrolled down is brought. At this time, I want to remove the information of the date that has already been compared. How?

### TRY 3
- 키워드 값이 없을때 자동으로 넘어가는 방법
- 던진 쿼리:
>* Q. NameError: name 'result_time' is not defined (에러 그대로 복사&붙여넣기)

### TRY4
- 스크롤이 멈춰있으면 넘어가도록 추가
>* Q. Can you write code to break the "while" when scrolling stops?

- 검색어 위치가 중간에 바뀜 --> try except 추가 (이건 본인이 추가)
- 최신순 클릭 추가 (이건 본인이 추가)

이외의 자잘한 에러들의 경우 그때그때 물어봐서 코드 수정


### ChatGPT에 대한 나의 생각 (My Opinion on Chatgpt)
- <span style="color:blue">Advantages</span>
- 확실히 코딩을 할때 수월했고 시간도 많이 단축되었다 (ChatGPT 덕분에 하루가 꼬박 걸렸을 복잡한 크롤링을 단 3시간만에 해결했다).
- 오류 자체를 복붙해서 물어봐도 잘 해결해준다.

- <span style="color:red">disadvantage</span>
- 하지만 오류가 많긴하다. 다시 본인이 고쳐야할 부분이 많다.
- 질문이 구체적일수록 답변의 정확도가 높다.

<span style='background-color:#fff5b1'>그럼에도 정말 효율적인 도구이다. 약간 과외 선생님 느낌이랄까? 하지만 동시 사용자가 많을 경우 멈춤현상이 있다....구독을 해야하나 고민중이다.</span>

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
import time
import os

import pandas as pd
from random import *

import warnings
warnings.filterwarnings(action='ignore') 

## Keyword

In [2]:
def get_keyword(master_keywords, K):
    text=master_keywords.split(" ")
    text_=K.split(" ")
    result=""
    for i in range(len(text_)):
        for j in range(len(text)):
            a=text_[i]+"+"+text[j] 
            result += a+","
    return result[:-1]


#마스터 키워드 (lifestyle 예시)
keyword='전기차 홈캠 AI스피커 로봇청소기 가스검침 nugu' # 공백으로 분리해주세요

#절대 키워드 (persona 예시)
KEY='시니어 액티브시니어 은퇴계획 황혼육아' 

final_keyword=get_keyword(keyword,KEY)

# Url list

In [3]:
url_list=[]
result_time=None #아무것도 없을 경우 자동 넘김

driver = webdriver.Chrome(ChromeDriverManager().install())

for i in final_keyword.split(","):
    driver.get(url='https://brunch.co.kr/')
    driver.maximize_window() # 돋보기 누르려면 크게 보기
    driver.implicitly_wait(2)
   
    try:
        search_box = driver.find_element_by_xpath('//*[@id="btnServiceMenuSearch"]')
        search_box.click()   
    except:
        pass
    
    search_bar=driver.find_element_by_xpath('//*[@id="txt_search"]')
    search_bar.send_keys(i)
    search_bar.send_keys(Keys.RETURN)
        
    #최신순 클릭
    try:
        a=driver.find_elements_by_css_selector('a.link_option')
        a[1].click()
    except:
        pass
        
    # create a set to store the dates
    date_set = set()
    
    # scroll down until the page
    #bruch shows the contents from current time
    target_date = datetime(2022, 1, 1)  # set the target date
    scroll_pause_time = 3  # set the pause time between scrolls
    current_time = datetime.now()
    

    # scroll until the target date is found
    while True:
        # scroll down to the bottom of the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        # find the latest post date
        latest_date = driver.find_elements_by_css_selector("span.mobile_d_n.post_append")
        for da in latest_date:
            publish_time = da.find_element_by_class_name("publish_time")
            pu_da=publish_time.text
                
            # change the date format to %y-m-d
            if len(pu_da)<6:
                num = int(''.join(filter(str.isdigit, pu_da)))
                n_hour_ago = current_time - timedelta(hours=int(num))
                result_time = n_hour_ago.strftime('%Y-%m-%d')
                result_time = datetime.strptime(result_time, '%Y-%m-%d')
            else: 
                parsed_date = datetime.strptime(pu_da, '%b %d. %Y')
                result_time = datetime.strptime(parsed_date.strftime('%Y-%m-%d'), '%Y-%m-%d')
    
            # check if the latest post date is earlier than the target date
            new_height = driver.execute_script("return document.body.scrollHeight")
            if result_time is None or result_time < target_date or new_height == last_height:
                break
            elif result_time not in date_set:
                date_set.add(result_time)
        
        time.sleep(0.8)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # break if the target date is reached
        if result_time is None or result_time < target_date or new_height == last_height:
            break
            
  

    # find the links on the page
    links = driver.find_elements_by_css_selector(".link_post")
    user = [link.get_attribute("href") for link in links]
    url_list.extend(user)
    print(f"{i}"+'url갯수: ', len(user))

# extract the URLs from the links and save to a list
df = pd.DataFrame({"URL": url_list})
df.to_excel(f"{KEY}_브런치_url_list.xlsx", index=False)
df



====== WebDriver manager ======
Current google-chrome version is 111.0.5563
Get LATEST chromedriver version for 111.0.5563 google-chrome
Driver [/Users/heojiyeon/.wdm/drivers/chromedriver/mac64/111.0.5563.64/chromedriver] found in cache


시니어+전기차url갯수:  40
시니어+홈캠url갯수:  0
시니어+AI스피커url갯수:  20
시니어+로봇청소기url갯수:  0
시니어+가스검침url갯수:  0
시니어+nuguurl갯수:  0
액티브시니어+전기차url갯수:  0
액티브시니어+홈캠url갯수:  0
액티브시니어+AI스피커url갯수:  0
액티브시니어+로봇청소기url갯수:  0
액티브시니어+가스검침url갯수:  0
액티브시니어+nuguurl갯수:  0
은퇴계획+전기차url갯수:  0
은퇴계획+홈캠url갯수:  0
은퇴계획+AI스피커url갯수:  0
은퇴계획+로봇청소기url갯수:  0
은퇴계획+가스검침url갯수:  0
은퇴계획+nuguurl갯수:  0
황혼육아+전기차url갯수:  0
황혼육아+홈캠url갯수:  0
황혼육아+AI스피커url갯수:  0
황혼육아+로봇청소기url갯수:  0
황혼육아+가스검침url갯수:  0
황혼육아+nuguurl갯수:  0


,URL
0,https://brunch.co.kr/@@2T9h/7
1,https://brunch.co.kr/@@5KoQ/521
2,https://brunch.co.kr/@@c0ua/32
3,https://brunch.co.kr/@@ekL2/22
4,https://brunch.co.kr/@@dZ2l/13
5,https://brunch.co.kr/@@2RHK/186
6,https://brunch.co.kr/@@cjHM/76
7,https://brunch.co.kr/@@cjHM/71
8,https://brunch.co.kr/@@bxXe/203
9,https://brunch.co.kr/@@cXD7/448


# Scraping titles, content, and comments

In [4]:
keyword=[]
Content=[] #제목+내용+댓글
Date=[]
com=""
con=""

current_time=datetime.now() 
    
#크롤링 페이지 카운트
count=0
for i in url_list:
    driver.get(i)
    
    # find the elements containing the title, contents, and date information
    title = driver.find_elements_by_class_name("cover_title")[0].text
    
    #date
    date = driver.find_elements_by_class_name("f_l.date")[0].text
    if len(date)<6:
        num = int(''.join(filter(str.isdigit, date)))
        n_hour_ago = current_time - timedelta(hours=int(num))
        result_time = n_hour_ago.strftime('%Y-%m-%d')
    
    else: 
        parsed_date = datetime.strptime(date, '%b %d. %Y')
        result_time=parsed_date.strftime('%Y-%m-%d')
   
    #click comment button
    try:
        comment_button = driver.find_element_by_css_selector('button.btn_comment')
    except:
        pass
    
    #그림,사진만 있을 경우
    try:
        content = driver.find_elements_by_class_name("wrap_item.item_type_text")
        for i in content:
            if i.text.strip():
                con+=i.text.strip() 
                
        comments = comment_section.find_elements_by_css_selector('ul.list_comment > li')
        for i in comments:
            com+=i.text
        
    except:
        content=""
        comment=""
    
    Content.append(title+', '+con+','+com)
    Date.append(result_time)
    count+=1
    time.sleep(uniform(0.5,0.7))
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [5]:
driver.quit()

In [6]:
# 데이터프레임에 저장할 키워드 
KEYWORD='그림홈테크'

final=pd.DataFrame({'review':Content, 'date': Date})
final['keyword']=f'{KEYWORD}'
final.to_excel(f"시니어_{KEYWORD}_브런치.xlsx", index=False)
final

,review,date,keyword
0,"빅데이터를 활용한 2023 관광트렌드, 빅데이터를 활용한 2023 관광트렌드지난 1...",2023-03-04,그림홈테크
1,"2023 주목할 기업 '종합' 부문 TOP 20, 빅데이터를 활용한 2023 관광트...",2023-02-22,그림홈테크
2,"여자의 바람 & 남자의 묘책, 빅데이터를 활용한 2023 관광트렌드지난 1월 31일...",2023-01-25,그림홈테크
3,"먹는다는 것, 빅데이터를 활용한 2023 관광트렌드지난 1월 31일 한국관광공사에서...",2023-01-24,그림홈테크
4,"지금이 좋다., 빅데이터를 활용한 2023 관광트렌드지난 1월 31일 한국관광공사에...",2022-12-21,그림홈테크
5,"내러티브 경제학2, 빅데이터를 활용한 2023 관광트렌드지난 1월 31일 한국관광공...",2022-12-10,그림홈테크
6,"투자사 회원이 가장 많이 조회한 신규등록기업 HOT 5, 빅데이터를 활용한 2023...",2022-11-21,그림홈테크
7,"3분기 거래액 월평균 성장률이 30% 이상인 스타트업, 빅데이터를 활용한 2023 ...",2022-11-02,그림홈테크
8,"소설 환취 (24화/25화), 빅데이터를 활용한 2023 관광트렌드지난 1월 31일...",2022-10-30,그림홈테크
9,"""역대급""인데 국내 출시 못하고 있는 현대차 정체?, 빅데이터를 활용한 2023 관...",2022-10-12,그림홈테크
